In [1]:
# To read the secret keys for Authentication
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

# To create new project
from azure.ai.language.questionanswering.authoring import AuthoringClient

# to create a question-answering client, and to ask questions using the knowledge base
from azure.ai.language.questionanswering import QuestionAnsweringClient
from azure.ai.language.questionanswering import models as qna

In [2]:
# get service secrets
load_dotenv()
endpoint = os.environ.get("endpoint")
key = os.environ.get("subscription_key")

In [3]:
# Create new project, add knowledge base and deploy it.
with AuthoringClient(endpoint, AzureKeyCredential(key)) as authoring_client:

    # Step 1: create project
    print("\n********************** Creating a new project *********************")
    project_name = "ICICIBank-LoanFAQs"
    project = authoring_client.create_project(
        project_name=project_name,
        options={
            "description": "FAQs related to ICICI banking loan system for Personal, Home and Car Loan.",
            "language": "en",
            "multilingualResource": True,
            "settings": {
                "defaultAnswer": "I'm sorry, I don't have the information to answer that right now."
            },
        },
    )

    # Output 1: View the project details
    print("view created project info:")
    print("\tname: {}".format(project["projectName"]))
    print("\tlanguage: {}".format(project["language"]))
    print("\tdescription: {}".format(project["description"]))

    # Step 2: Add a knowledge base
    print("\n********************* Adding a knowledge base *********************")

    update_sources_poller = authoring_client.begin_update_sources(
        project_name=project_name,
        sources=[
            {
                "op": "add",
                "value": {
                    "displayName": "Personal Loan FAQ",
                    "sourceUri": "https://www.icicibank.com/personal-banking/loans/personal-loan/personal-loans-faqs",
                    "sourceKind": "url",
                },
            },
            {
                "op": "add",
                "value": {
                    "displayName": "Home Loan FAQ",
                    "sourceUri": "https://www.icicibank.com/personal-banking/loans/home-loan/home-loans-faqs",
                    "sourceKind": "url",
                },
            },
            {
                "op": "add",
                "value": {
                    "displayName": "Car Loan FAQ",
                    "sourceUri": "https://www.icicibank.com/personal-banking/loans/car-loan/car-loans-faqs",
                    "sourceKind": "url",
                },
            },
        ],
    )
    update_sources_poller.result()

    # Output 2: list sources
    print("\nlist project sources")
    sources = authoring_client.list_sources(project_name=project_name)
    for source in sources:
        print("knowledge base name: {}".format(source["displayName"]))
        print("\tsource: {}".format(source["source"]))
        print("\tsource Uri: {}".format(source["sourceUri"]))
        print("\tsource kind: {}".format(source["sourceKind"]))

    # Step 3: deploy the project
    print("\n********************** Deploying the project **********************")

    deployment_poller = authoring_client.begin_deploy_project(
        project_name=project_name, deployment_name="production"
    )
    deployment_poller.result()

    # list all deployments
    deployments = authoring_client.list_deployments(project_name=project_name)

    print("view project deployments")
    for d in deployments:
        print(d)


********************** Creating a new project *********************


ClientAuthenticationError: (401) Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.
Code: 401
Message: Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.

In [16]:
def ask_question(question):
    with QuestionAnsweringClient(endpoint, AzureKeyCredential(key)) as qna_client:
        output = qna_client.get_answers(
            question=question,
            top=3,
            confidence_threshold=0.5,
            include_unstructured_sources=True,
            short_answer_options=qna.ShortAnswerOptions(
                confidence_threshold=0.5, top=1
            ),
            project_name=project_name,
            deployment_name="production",
        )
        answers = (
            [a for a in output.answers if a.confidence and a.confidence > 0.5]
            if output.answers
            else []
        )
        if len(answers) > 0:
            best_candidate = answers[0]
            print(f"Q: {question}")
            print(f"A: {best_candidate.answer}")
        else:
            print(f"No answers returned from question '{question}'")

In [17]:
# 1. Test by asking a question
ask_question("personal loan interest rate?")

Q: personal loan interest rate?
A: Personal Loan interest rates are applied basis the defined pricing matrix followed and standardised by the bank as per customer’s profile. Personal Loan rates vary from 10.80% to 16.15% per annum.


In [18]:
# 2. Test by asking a non relevant question
ask_question("offer me tea in bank?")

No answers returned from question 'offer me tea in bank?'
